In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [106]:
# Read the provided CSV files with 'cp949' encoding
park_east_df = pd.read_csv("데이터/데이터/2. 가까워야하는 공공시설/공원_동구.csv", encoding='cp949')
park_middle_df = pd.read_csv("데이터/데이터/2. 가까워야하는 공공시설/공원_중구.csv", encoding='cp949')
target_df = pd.read_csv("target.csv", encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: '데이터/데이터/2. 가까워야하는 공공시설/공원_동구.csv'

In [ ]:
# 중구 동구 공원데이터 합치기
park_df = pd.concat([park_east_df,park_middle_df])

In [ ]:
park_df

In [ ]:
# 공원과 학교간의 거리를 측정하기 위한 함수 생성
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    # 지구 반지름을 이용한 거리 계산     
    # 경위도 라디안으로 변경
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula 
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a)) 
    distance = 6371 * c # Radius of earth in kilometers
    return distance

def nearest_park_distance(target_lat, target_lon, park_df):
    distances = park_df.apply(lambda row: haversine_distance(target_lat, target_lon, row['위도'], row['경도']), axis=1)
    return distances.min()

    

In [ ]:
# 지표 1 : 공원과의 거리
# 동네 별 인구 수 비교, target의 좌표들이 어느 동네에 속하는 지 확인 인구 수가 높은 동네에 속하는 target일수록 높은 등수, 같은 동네 같은 등수?
p_d_list =[]
for idx, row in target_df.iterrows():
    target_lon = row['X']
    target_lat = row['Y']
    d =nearest_park_distance(target_lat, target_lon, park_df)
    p_d_list.append(d)

In [ ]:
len(d_list)

In [ ]:
mean_value = np.mean(d_list)
median_value = np.median(d_list)
std_dev = np.std(d_list)
min_value = np.min(d_list)
max_value = np.max(d_list)
percentiles = [np.percentile(d_list, p) for p in [25, 50, 75]]

# Plotting histogram
plt.hist(d_list, bins=30, edgecolor='k', alpha=0.65)
plt.axvline(mean_value, color='r', linestyle='dashed', linewidth=1, label=f"Mean: {mean_value:.2f}")
plt.axvline(median_value, color='y', linestyle='dashed', linewidth=1, label=f"Median: {median_value:.2f}")
plt.title('Histogram of Sample Data')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()


In [ ]:
n_df = pd.read_csv('시설 정보 현황.csv',encoding='cp949')

In [ ]:
n_df['유형'].unique()

In [ ]:
# 법령에 근거해서 도시숲, 수변공원, 체육시설의 위치와 target간의 거리 확인
f_df = n_df[n_df['유형'] == '도시숲']
w_df = n_df[n_df['유형'] == '수변공원']
g_df = n_df[n_df['유형'] == '체육시설']

In [ ]:
f_d_list =[]
for idx, row in target_df.iterrows():
    target_lon = row['X']
    target_lat = row['Y']
    d =nearest_park_distance(target_lat, target_lon, f_df)
    f_d_list.append(d)

In [ ]:
w_d_list =[]
for idx, row in target_df.iterrows():
    target_lon = row['X']
    target_lat = row['Y']
    d =nearest_park_distance(target_lat, target_lon, w_df)
    w_d_list.append(d)

In [ ]:
g_d_list =[]
for idx, row in target_df.iterrows():
    target_lon = row['X']
    target_lat = row['Y']
    d =nearest_park_distance(target_lat, target_lon, g_df)
    g_d_list.append(d)

In [ ]:
df = pd.DataFrame({'숲' : f_d_list,'체육시설': g_d_list})

In [ ]:
df

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

In [ ]:
plt.scatter(df.values[:, 0], df.values[:, 1])

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.75, marker='X')
plt.title("K-means Clustering")
plt.show()

In [ ]:
assignments_X_train = kmeans.labels_
print(assignments_X_train)

In [ ]:
idx = []
for i in range(len(assignments_X_train)):
    if assignments_X_train[i] == 0 :
        idx.append(i)
print(idx)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

# AgglomerativeClustering 객체 생성
agg_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')

# 군집화 수행
clusters = agg_cluster.fit_predict(df)

# 결과 시각화
plt.scatter(df.iloc[:, 0], df.iloc[:, 1], cmap='viridis')
plt.title("Agglomerative Clustering")
plt.show()

# 덴드로그램으로 군집 구조 시각화
def plot_dendrogram(model, **kwargs):
    children = model.children_
    distance = np.arange(children.shape[0])
    position = np.arange(children.shape[0])
    linkage_matrix = np.column_stack([children, distance, position]).astype(float)
    dendrogram(linkage_matrix, **kwargs)

plt.figure(figsize=(10, 5))
plot_dendrogram(agg_cluster, truncate_mode='level', p=3)
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

In [ ]:
labels = agg_cluster.labels_
print(labels)

In [ ]:
idx2 = []
for i in range(len(labels)):
    if labels[i] == 0 :
        idx2.append(i)
print(idx2)

In [ ]:
for i in idx:
    if i in idx2:
        print('T')
    else:
        print('F')